In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pip install soupsieve
!pip install fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=685b5fb3d8368b416ba75656bb681ad1bd0ea887d41cef2f5897b2d0deaa09f1
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [3]:
import requests
import time
import random
import decimal
import json
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

In [4]:
words = []
filetypes=[]

In [5]:
while True:
    word = input("Enter Word Or Exit: ")
    if word.lower() == "exit":
        break
    words.append(word)

Enter Word Or Exit: python
Enter Word Or Exit: scrapy
Enter Word Or Exit: exit


In [6]:
while True:
    filetype = input("Enter FileType Or Exit: ")
    if filetype.lower() == "exit":
        break
    filetypes.append(filetype)

Enter FileType Or Exit: pdf
Enter FileType Or Exit: exit


In [8]:
for filetype in filetypes:
  for word in words:
    requests.packages.urllib3.disable_warnings()
    useragent = UserAgent()
    results_obj = {}
    referrer = 'https://ogs.google.com/'
    page = 1
    startnum = 0
    list1=[]
    flag = True
    textdork_str=str(word)+'+filetype:'+str(filetype)
    output_path="/content/drive/My Drive/"+str(word)+"_"+str(filetype)+".json"

    print("Dork: "+textdork_str+"  \n   Output path:"+output_path)

    try:
      while flag:
          print("{0} page".format(page))
          session = requests.session()
          response = session.get('http://google.com')
          cookies = session.cookies.get_dict()
          headers = {'user-agent': useragent.random,
                    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                    'Referer': referrer,
                    'Upgrade-Insecure-Requests': '1'
                    }
          for k, v in cookies.items():
              headers[k] = v
          url = 'https://www.google.com/search?q=' + textdork_str
          url = url + '&num=100' '&start=' + str(startnum) + '&tbas=0'
          req = requests.get(url, headers=headers, verify=False)
          soup = BeautifulSoup(req.content, "html.parser")
          resultbody = soup.find('div', {'id': 'search'})
          links = resultbody.find_all('a', href=True)
          list1.append(links)
          if len(resultbody) >= 1:
              results = []
              for l in links:
                  if 'webcache' not in l['href'] and '#' not in l['href'] and '?q=related:' not in l['href']:
                      results.append(l['href'])
              sections = resultbody.find_all('div', {'class': 'rc'})

              for section in sections:
                  href = section.find('a')['href']
                  parsed_domain = urlparse(href)
                  domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_domain)
                  title = section.find('h3').text
                  desc = section.find('span', {'class': 'st'}).text

                  info = {'URL': href,
                          'Title': title,
                          'Text': desc,
                          'Page': page
                          }

                  results_obj[domain] = info
          else:
              flag = False

          if flag == True:
              page += 1
              
          startnum += 100
          referrer = str(url)
          time.sleep(12)
      print("Writing to Json File")
      with open(output_path, 'w') as outfile:
          json.dump(results_obj, outfile)

    except:
      print("Error")
      print("Writing to Json File")
      with open(output_path, 'w') as outfile:
          json.dump(results_obj, outfile)
      


Dork: python+filetype:pdf  
   Output path:/content/drive/My Drive/python_pdf.json
1 page
2 page
3 page
4 page
Writing to Json File
Dork: scrapy+filetype:pdf  
   Output path:/content/drive/My Drive/scrapy_pdf.json
1 page
2 page
3 page
Writing to Json File


**Note:**
If you get  "Error", you must end the session from Manage Session and 
start it again from the beginning.



